In [ ]:
import cv2
import numpy as np
from skimage.measure import structural_similarity as ssim

def mse (image1, image2):
    error = np.sum((image1.astype('float') - image2.astype('float'))**2)
    #We assume there are the same number of pixels in the images
    error = error/float(image1.shape[0]*image1.shape[1])
    return error

def ssimCompare (image1, image2):
    #Need to fix this!
    return ssim(image1, image2)

def normalRebuild(frameCount):
    differenceCount = 1
    #Create Empty List
    frameDiffList = [None] * (frameCount -1)
    #Find Difference and Fill List
    while differenceCount < frameCount -1:
        #open two images
        image1 = cv2.imread("frame%d.jpeg" %differenceCount)
        image2 = cv2.imread("frame%d.jpeg" %(differenceCount+1)) 
        #call mse
        imageDiff = mse(image1, image2)
        #store result in list
        frameDiffList[differenceCount] = imageDiff    
        #repeat
        differenceCount +=1
    print "Similarity metric computed."
    #Rebuilding a video based on any frame within threshold of similarity metric
    rebuildIndex = 0
    print "Now Building Video"
    sampleimageshape = cv2.imread('frame1.jpeg')
    height, width, layers = sampleimageshape.shape
    video = cv2.VideoWriter('new.avi', 0, 30.0, (width, height))
    while rebuildIndex < frameCount-1:
        #run through list and if the threshold is less than something we add to the new movie
        if(frameDiffList[rebuildIndex]<10):
            video.write(cv2.imread('frame%d.jpeg' %(rebuildIndex+1)))
            #print "frames writen frame%d.jpeg" %(rebuildIndex+1)
            video.write(cv2.imread('frame%d.jpeg' %(rebuildIndex+2)))
            #print "frames writen frame%d.jpeg" %(rebuildIndex+2)
        rebuildIndex +=2
    print "All frames writen"
    video.release()
    print "Video released"
    cv2.destroyAllWindows()
    print "All Windows Destroyed"
    
def frameTargetRebuild(frameCount, naiveTarget, naiveDeviation):
    #Rebuild Video Based on Specified Deviation from Naive Time Lapse
    print "Now Building Video"
    if((naiveDeviation > naiveTarget) or (naiveTarget>frameCount)):
        print"Allowable deviation must be less than the target"
        return
    sampleimageshape = cv2.imread('frame1.jpeg')
    height, width, layers = sampleimageshape.shape
    video = cv2.VideoWriter('new.avi', 0, 30.0, (width, height))
    video.write(cv2.imread('frame1.jpeg'))
    #Need a Target, Start with first frame find closest within deviation from naive
    loopCount = 1
    while(loopCount*naiveTarget < frameCount):
        if(loopCount<2):
            image1 = cv2.imread("frame1.jpeg")
        else:
            #image is last writen image
            image1 = cv2.imread("frame%d.jpeg", oldFrameNum)
        #Initial Compare
        frameDiffList = [None] * ((naiveDeviation *2)+1)
        compareCount = 0
        while(compareCount < ((naiveDeviation * 2)+1)-1):
            #Create an array of the differences between last frame and all possible second images
            #There's an error here with the frame number index calc, this must be fixed
            image2 = cv2.imread("frame%d.jpeg" %(((loopCount*naiveTarget)-naiveDeviation)+compareCount))
            print "frame number %d" %(((loopCount*naiveTarget)-naiveDeviation)+compareCount)
            imageDiff=mse(image1, image2)
            frameDiffList[compareCount] = imageDiff
            compareCount+=1   
        #Find min value in array, get corresponding image and  add to video then save image writen as image1 for next loop
        index_min = np.argmin(frameDiffList)
        video.write(cv2.imread("frame%d.jpeg" %(((loopCount*naiveTarget)-naiveDeviation)+index_min)))
        #Update Counters and assign value to old frame num for future comparisons
        oldFrameNum = (((loopCount*naiveTarget)-naiveDeviation)+index_min)
        loopCount +=1  
    print "All frames writen"
    video.release()
    print "Video released"
    cv2.destroyAllWindows()
    print "All Windows Destroyed"

#Main Function
#Call a rebuilding function with a frame count AND a similarity metric as a parameter
#normalRebuild(166)
frameTargetRebuild(166, 5, 2)
